<a href="https://colab.research.google.com/github/abhinesh-kourav/langchain_pinecone_llama3/blob/main/Pinecone_with_Llama_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pinecone: https://www.pinecone.io/

In [1]:
!pip install langchain
!pip install pinecone-client --upgrade
!pip install pypdf

In [2]:

!pip install sentence-transformers

In [3]:
!mkdir pdfs

mkdir: cannot create directory ‘pdfs’: File exists


## Extract the Text from the PDF's

In [4]:
!pip install -U langchain-community

In [5]:
from langchain.document_loaders import PyPDFDirectoryLoader

In [6]:
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()

In [7]:
data

[Document(metadata={'source': 'pdfs/1706.03762v7.pdf', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transf

## Chunkins

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

In [10]:
len(text_chunks)

91

In [11]:
text_chunks[10]

Document(metadata={'source': 'pdfs/1706.03762v7.pdf', 'page': 1}, page_content='the input or output sequences [ 2,19]. In all but a few cases [ 27], however, such attention mechanisms\nare used in conjunction with a recurrent network.\nIn this work we propose the Transformer, a model architecture eschewing recurrence and instead\nrelying entirely on an attention mechanism to draw global dependencies between input and output.\nThe Transformer allows for significantly more parallelization and can reach a new state of the art in')

## Embeddings

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased")

<ipython-input-13-25881b8bbe68>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This b

In [14]:
query_result = embeddings.embed_query("Hello World")

In [15]:
query_result

[0.029456475749611855,
 0.004736720584332943,
 -0.04529789835214615,
 -0.0019095977768301964,
 0.00021328963339328766,
 -0.04217856749892235,
 0.03153294697403908,
 -0.01571282371878624,
 -0.04520609974861145,
 -0.01053853053599596,
 0.008551565930247307,
 -0.06312336027622223,
 0.03493773564696312,
 0.04391759634017944,
 -0.03331681340932846,
 0.00019311998039484024,
 0.021816274151206017,
 0.03053576499223709,
 0.06767795234918594,
 -0.024694407358765602,
 0.032428909093141556,
 0.009735566563904285,
 0.06773286312818527,
 0.01718902960419655,
 0.024372830986976624,
 -0.016025559976696968,
 0.05916434898972511,
 -0.02172166481614113,
 -0.035340532660484314,
 0.0010092627489939332,
 0.02185635454952717,
 -0.04162004590034485,
 -0.05435357242822647,
 -0.07162235677242279,
 0.03782526031136513,
 -0.04609986022114754,
 -0.035168398171663284,
 0.002208614954724908,
 0.015811672434210777,
 -0.07625781744718552,
 0.031209411099553108,
 -0.014202098362147808,
 -0.03857778385281563,
 0.049565

In [16]:
print("Length", len(query_result))

Length 512


## Initializing the Pinecone

In [17]:
import os

In [18]:
from google.colab import userdata
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')

In [19]:
from pinecone import Pinecone
pc = Pinecone(PINECONE_API_KEY)

In [20]:
index_name = "check"
index = pc.Index(index_name)

In [21]:
!pip install -qU langchain-pinecone pinecone-notebooks

In [22]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

## Similarity Search

In [23]:
query = "What is attention?"

In [24]:
docs = vector_store.similarity_search(query, k=3)

In [25]:
docs

[Document(id='3278e67c-159b-4073-ad13-7b389cb982b3', metadata={'page': 5.0, 'source': 'pdfs/oreilly_chapter_excerpt_nlpt.pdf'}, page_content='79Attention Please!                                                                                                           81Sparse Attention                                                                                                           82Linearized Attention                                                                                                    83Going Beyond Text'),
 Document(id='c556d57c-21a7-4a56-a3f8-652a10d07e3d', metadata={'page': 87.0, 'source': 'pdfs/oreilly_chapter_excerpt_nlpt.pdf'}, page_content='dilated window with gapsRandom attentionRandomly samples a few keys for each query to compute attention scores'),
 Document(id='14066ea5-5e4a-4e72-b784-09c87ea70c4f', metadata={'page': 87.0, 'source': 'pdfs/oreilly_chapter_excerpt_nlpt.pdf'}, page_content='Figure 11-5. Common atomic sparse attention patterns for self-

In [26]:
len(docs)

3

In [27]:
from langchain.chains import RetrievalQA

In [28]:
import torch
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

In [29]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

from huggingface_hub import login
hf_token = userdata.get('HF_TOKEN')
login(token=hf_token)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [30]:
!pip install -U bitsandbytes

In [31]:
# Configure bitsandbytes quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Use 4-bit quantization for faster inference
    bnb_4bit_compute_dtype="float16"
)

In [32]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_new_tokens=500,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [33]:
llm = HuggingFacePipeline(pipeline=pipeline)

<ipython-input-33-eff6020df754>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipeline)


In [34]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [35]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever())

In [38]:
# full example
query = "Why attention is all I need"
llm_response = qa(query)
llm_response

{'query': 'Why attention is all I need',
 'result': 'Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\n3A. Vaswani et al., “ Attention Is All Y ou Need”, (2017).\n\ndilated window with gapsRandom attentionRandomly samples a few keys for each query to compute attention scores\n\n79Attention Please!                                                                                                           81Sparse Attention                                                                                                           82Linearized Attention                                                                                                    83Going Beyond Text\n\nhead. The resulting multi-head attention layer isillustrated in Figure 3-5. But why do we need more than one attention head? The rea/uni2010son is that the softmax of one head tends to focus on mostly one a